In [1]:
from typing import List, Callable, Union, Any, TypeVar, Tuple
from torch import nn
import torch
from torch.nn import functional as F
import numpy as np
import pandas as pd
from torch import tensor as Tensor
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import random
from models.rnn import CMV_LSTM, MV_LSTM
from models.vae import ConditionalVAE, VanillaVAEEncoder, VanillaVAEDecoder
from models.rvae import UnitRVAE, RVAE
from utils.loss import VAE_Loss

In [2]:
seq_len = 5
batch_size = 4
torch.autograd.set_detect_anomaly(True)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
def get_total_datas(seq_length, data, shuffle=True):
    total_num = len(data)
    list_x_datas = []
    list_y_datas = []
    for i in range(seq_length, total_num):
        list_y_datas.append(data[i].reshape([1, *data[i].shape]))
        list_x_datas.append(data[i - seq_length:i].copy().reshape([1, *data[i - seq_length:i].shape]))
    
    if shuffle:
        suffled_idices = list(range(len(list_x_datas)))
        random.shuffle(suffled_idices)
        list_x_datas1 = []
        list_y_datas1 = []
        for i in suffled_idices:
            list_x_datas1.append(list_x_datas[i])
            list_y_datas1.append(list_y_datas[i])
        return np.vstack(list_x_datas1), np.vstack(list_y_datas1)
    return np.vstack(list_x_datas), np.vstack(list_y_datas)


class timeseries(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.len = x.shape[0]

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
  
    def __len__(self):
        return self.len

In [7]:
df_econs = pd.read_csv('data/total_economy_data.csv', index_col=0)
df_econs1 = df_econs.drop(columns=['year', 'month'])
df_econs1 = df_econs1.dropna(axis=1)
arr_econs = df_econs1.to_numpy()
arr_econs = arr_econs / 1e10

In [8]:
# Pretrain

train_data = arr_econs
urvae = UnitRVAE(arr_econs.shape[-1], seq_len, 28)

x_train, y_train = get_total_datas(seq_len, train_data, shuffle=True)
ts_dataset = timeseries(x_train, y_train)


model = urvae.to(device)
urvae.rnn.init_hidden(batch_size)
criterion = VAE_Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
epochs = 50

for i in range(epochs):
    ts_train_loader = DataLoader(ts_dataset, shuffle=True, batch_size=batch_size)
    model.train()
    for j, data in enumerate(ts_train_loader):
        if data[0].shape[0] != batch_size:
            break
        curr_data = data[0].clone().detach(), data[1].clone().detach()
        output = model(curr_data[0])
        loss = criterion(output[2], output[3], output[0], curr_data[1])
        #make_dot(loss, params=dict(model.named_parameters())).render(f"graph", format="png")
        #result = model.loss_function(curr_data[1].clone(), *output, **{'M_N': 1})
        #loss = result['loss']
        optimizer.zero_grad()
        loss.backward(retain_graph=False)
        optimizer.step()

        
    if i % 2 == 0:
        print(i, 'th iteration : ', loss)

0 th iteration :  tensor(30.8104, grad_fn=<AddBackward0>)
2 th iteration :  tensor(32.5572, grad_fn=<AddBackward0>)
4 th iteration :  tensor(38.4819, grad_fn=<AddBackward0>)
6 th iteration :  tensor(43.9157, grad_fn=<AddBackward0>)
8 th iteration :  tensor(23.6326, grad_fn=<AddBackward0>)
10 th iteration :  tensor(29.4062, grad_fn=<AddBackward0>)
12 th iteration :  tensor(30.8977, grad_fn=<AddBackward0>)
14 th iteration :  tensor(14.0472, grad_fn=<AddBackward0>)
16 th iteration :  tensor(18.6920, grad_fn=<AddBackward0>)
18 th iteration :  tensor(17.5703, grad_fn=<AddBackward0>)
20 th iteration :  tensor(26.2875, grad_fn=<AddBackward0>)
22 th iteration :  tensor(18.1912, grad_fn=<AddBackward0>)
24 th iteration :  tensor(15.2267, grad_fn=<AddBackward0>)
26 th iteration :  tensor(15.6488, grad_fn=<AddBackward0>)
28 th iteration :  tensor(20.5717, grad_fn=<AddBackward0>)
30 th iteration :  tensor(28.4618, grad_fn=<AddBackward0>)
32 th iteration :  tensor(29.1979, grad_fn=<AddBackward0>)
34

In [ ]:
x_train, y_train = get_total_datas(seq_len, train_data, shuffle=True)
ts_dataset = timeseries(x_train, y_train)

x_test, y_test = get_total_datas(seq_len, test_data, shuffle=True)
test_dataset = timeseries(x_test, y_test)

model = urvae.to(device)
urvae.rnn.init_hidden(batch_size)
criterion = VAE_Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 50

for i in range(epochs):
    ts_train_loader = DataLoader(ts_dataset, shuffle=True, batch_size=batch_size)
    test_train_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)
    model.train()
    for j, data in enumerate(ts_train_loader):
        if data[0].shape[0] != batch_size:
            break
        curr_data = data[0].clone().detach(), data[1].clone().detach()
        output = model(curr_data[0])
        loss = criterion(output[2], output[3], output[0], curr_data[1])
        #make_dot(loss, params=dict(model.named_parameters())).render(f"graph", format="png")
        #result = model.loss_function(curr_data[1].clone(), *output, **{'M_N': 1})
        #loss = result['loss']
        optimizer.zero_grad()
        loss.backward(retain_graph=False)
        optimizer.step()
    model.eval()
    for j, data in enumerate(test_train_loader):
        if data[0].shape[0] != batch_size:
            break
        curr_data = data[0].clone().detach(), data[1].clone().detach()
        output = model(curr_data[0])
        val_loss = criterion(output[2], output[3], output[0], curr_data[1])
        
    if i % 2 == 0:
        print(i, 'th iteration : ', val_loss)